# **Instructions**
It is best to to create a virtual environment.

We need to install the following libraries and dependencies<br>

Run the following commands:<br>
`pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118`<br>
`pip install torchmetrics`<br>
`pip install matplotlib`<br>

Download the **Imagenet2012 Validation dataset** and unzip it to the **"data"** directory.<br>

You might run into a problem when running the first cell. The following error might show up: **"cannot import name 'zero_gradients' from 'torch.autograd.gradcheck"**<br>
There will be a link at the end of the error, which should lead you to **gradcheck.py** file<br>
Copy and paste the following code the **gradcheck.py** file

```
def zero_gradients(x):
    if isinstance(x, torch.Tensor):
        if x.grad is not None:
            x.grad.detach_()
            x.grad.zero_()
    elif isinstance(x, collections.abc.Iterable):
        for elem in x:
            zero_gradients(elem)
```
Save the gradcheck.py file.

# **Import Libraries**

In [ ]:
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import numpy as np
import matplotlib.pyplot as plt
import torch.optim as optim
import torch.utils.data as data_utils
from torch.autograd import Variable
import math
import torchvision.models as models
from PIL import Image
from deepfool_targeted import deepfool_targeted
import os
import math
import numpy as np
import torch as torch
import copy
from torch.autograd.gradcheck import zero_gradients

import torch
import torchvision.datasets as datasets
from torchvision import transforms

import matplotlib.image as mpimg

import json
# import time

In [ ]:
# Used models
# models.resnet50(pretrained=True),
# models.alexnet(pretrained=True),
# models.efficientnet_v2_s(pretrained=True),
# models.googlenet(pretrained=True),
# models.inception_v3(pretrained=True),
# models.vit_b_16(pretrained=True)

Please, check the `deepfool_targeted.py` file to see if the needed function is commented out.

# **ResNet50**

In [ ]:
import time
start = time.process_time()
stats_dict = {}
stats_dict_temp = {}

# Storing true labels in an array
filename = './data/LOC_val_solution/ILSVRC2012_val_true_label_indices.txt'

with open(filename, 'r') as file:
    true_labels = []
    for line in file:
        true_labels.append(int(line.strip()))

net = models.resnet50(pretrained=True)

# Switch to evaluation mode
net.eval()

img_dir = './data/ILSVRC2012_img_val/'

files = sorted(os.listdir(img_dir)) # Use this line if you want to run for the whole dataset

# Select only the specific file names 
# [1000:] = start from 1000 and end at file end, 
# [:1000] = run from start till 1000th file
# [65:36000] = run from 66th file to 36000th file

# files = os.listdir(img_dir)[36287:] #Change the value inside [] according to need

confidences_pert = []
l2_norms = []
ssim_scores = []
index = 0
iterations = []
success = []

for filename in files:
    
    # Create the 'perturbed_images' directory if it doesn't exist
    if not os.path.exists('./data/perturbed_images_resnet50'):
        os.makedirs('./data/perturbed_images_resnet50')

    # Load the image
    im_orig = Image.open(os.path.join(img_dir, filename)).convert('RGB')
    # im_orig = Image.open(os.path.join(img_dir, filename))
    # if im_orig.mode == 'L': # grayscale image
    #     im_orig = im_orig.convert('RGB')

    mean = [ 0.485, 0.456, 0.406 ]
    std = [ 0.229, 0.224, 0.225 ]

    # Apply the transformation pipeline
    im = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean = mean,
                             std = std)])(im_orig)
    
    # randomly select target class
    target = np.random.choice([i for i in range(1000) if i != true_labels[index]])

    # Perform the targeted deepfool attack
    r, loop_i, label_orig, label_pert, pert_image, confidences, confidence_pert, l2_norm, ssim_value, successful = deepfool_targeted(im, net, target)

    confidences_pert.append(confidence_pert)
    l2_norms.append(l2_norm)
    ssim_scores.append(ssim_value)
    iterations.append(loop_i)
    success.append(successful)

    labels = open(os.path.join('synset_words.txt'), 'r').read().split('\n')
    
    # Get the string label for the original and perturbed images
    str_label_orig = labels[np.int_(label_orig)].split(',')[0]
    str_label_pert = labels[np.int_(label_pert)].split(',')[0]

    def clip_tensor(A, minv, maxv):
        A = torch.max(A, minv*torch.ones(A.shape))
        A = torch.min(A, maxv*torch.ones(A.shape))
        return A

    clip = lambda x: clip_tensor(x, 0, 255)

    tf = transforms.Compose([transforms.Normalize(mean=[0, 0, 0], std=list(map(lambda x: 1 / x, std))),
    transforms.Normalize(mean=list(map(lambda x: -x, mean)), std=[1, 1, 1]),
    transforms.Lambda(clip),
    transforms.ToPILImage(),
    transforms.CenterCrop(224)])

    # Clip the perturbed image and convert it back to a PIL image
    pert_image = tf(pert_image.cpu()[0])

    # Save the perturbed image
    pert_image.save(os.path.join('./data/perturbed_images_resnet50', filename))
    
    # Print the labels of the original and perturbed images
    print(f"{filename}: Original label = {str_label_orig}, Perturbed label = {str_label_pert}")
    
    index += 1

    # Add model name as key and corresponding values as values in stats_dict_temp
    # Update test_dict with the new values for alexnet
    # Check if 'resnet50' is already in stats_dict_temp
    if 'resnet50' in stats_dict_temp:
        # If it is, update the existing dictionary for 'resnet50'
        stats_dict_temp['resnet50'].update({
            'confidences_pert': stats_dict_temp['resnet50'].get('confidences_pert', []) + [confidence_pert],
            'l2_norms': stats_dict_temp['resnet50'].get('l2_norms', []) + [l2_norm],
            'ssim_scores': stats_dict_temp['resnet50'].get('ssim_scores', []) + [ssim_value],
            'iterations': stats_dict_temp['resnet50'].get('iterations', []) + [loop_i],
            'success': stats_dict_temp['resnet50'].get('success', []) + [successful]
        })
    else:
        # If it isn't, add a new dictionary for 'resnet50'
        stats_dict_temp['resnet50'] = {
            'confidences_pert': [confidence_pert],
            'l2_norms': [l2_norm],
            'ssim_scores': [ssim_value],
            'iterations': [loop_i],
            'success': [successful]
        }
    
    with open('./stats/resnet50_stats_temp.json', 'w') as f:
        json.dump(stats_dict_temp, f)

In [ ]:
time =  time.process_time() - start
print("Total Time: ",time)

In [ ]:
# Add model name as key and corresponding values as values in stats_dict
stats_dict.update({'resnet50': {
                            'confidences_pert': confidences_pert,
                            'l2_norms': l2_norms,
                            'ssim_scores': ssim_scores,
                            'iterations': iterations,
                            'success': success
                            }})

In [ ]:
# Write dictionary to file
with open('./stats/resnet50_stats.json', 'w') as f:
    json.dump(stats_dict, f)

In [ ]:
# Read dictionary from file
with open('./stats/resnet50_stats.json', 'r') as f:
    stats_dict = json.load(f)

model_name = "resnet50"
stats = stats_dict[model_name]

print(f"Mean Confidence for {model_name}: {sum(stats['confidences_pert'])/len(stats['confidences_pert']):.4f}")
print(f"Mean l2_norm for {model_name}: {sum(stats['l2_norms'])/len(stats['l2_norms']):.4f}%")
print(f"Mean SSIM score for {model_name}: {sum(stats['ssim_scores'])/len(stats['ssim_scores']):.4f}")
print(f"Mean Iterations for {model_name}: {sum(stats['iterations'])/len(stats['iterations']):.4f}")
print(f"Mean success for {model_name}: {sum(stats['success'])/len(stats['success']):.4f}\n")

In [ ]:
# Shows results after excluding failed attacks

with open('./stats/resnet50_stats.json', 'r') as f:
    stats_dict = json.load(f)

model_name = "resnet50"
stats = stats_dict[model_name]

success_indices = [i for i, success in enumerate(stats['success']) if success]

success_confidences = [stats['confidences_pert'][i] for i in success_indices]
success_l2_norms = [stats['l2_norms'][i] for i in success_indices]
success_ssim_scores = [stats['ssim_scores'][i] for i in success_indices]
success_iterations = [stats['iterations'][i] for i in success_indices]

print(f"Mean Confidence for {model_name}: {sum(success_confidences) / len(success_confidences):.4f}")
print(f"Mean l2_norm for {model_name}: {sum(success_l2_norms) / len(success_l2_norms):.4f}%")
print(f"Mean SSIM score for {model_name}: {sum(success_ssim_scores) / len(success_ssim_scores):.4f}")
print(f"Mean Iterations for {model_name}: {math.ceil(sum(success_iterations) / len(success_iterations))}\n")


# **AlexNet**

In [ ]:
import time
start = time.process_time()
stats_dict = {}
stats_dict_temp = {}
# Storing true labels in an array
filename = './data/LOC_val_solution/ILSVRC2012_val_true_label_indices.txt'

with open(filename, 'r') as file:
    true_labels = []
    for line in file:
        true_labels.append(int(line.strip()))

net = models.alexnet(pretrained=True)

# Switch to evaluation mode
net.eval()

img_dir = './data/ILSVRC2012_img_val/'

files = sorted(os.listdir(img_dir)) # Use this line if you want to run for the whole dataset

# Select only the specific file names 
# [1000:] = start from 1000 and end at file end, 
# [:1000] = run from start till 1000th file
# [65:36000] = run from 66th file to 36000th file

# files = os.listdir(img_dir)[36287:] #Change the value inside [] according to need

confidences_pert = []
l2_norms = []
ssim_scores = []
index = 0
iterations = []
success = []

for filename in files:
    
    # Create the 'perturbed_images' directory if it doesn't exist
    if not os.path.exists('./data/perturbed_images_alexnet'):
        os.makedirs('./data/perturbed_images_alexnet')

    # Load the image
    im_orig = Image.open(os.path.join(img_dir, filename)).convert('RGB')
    # im_orig = Image.open(os.path.join(img_dir, filename))
    # if im_orig.mode == 'L': # grayscale image
    #     im_orig = im_orig.convert('RGB')

    mean = [ 0.485, 0.456, 0.406 ]
    std = [ 0.229, 0.224, 0.225 ]

    # Apply the transformation pipeline
    im = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean = mean,
                             std = std)])(im_orig)
    
    # randomly select target class
    target = np.random.choice([i for i in range(1000) if i != true_labels[index]])

    # Perform the targeted deepfool attack
    r, loop_i, label_orig, label_pert, pert_image, confidences, confidence_pert, l2_norm, ssim_value, successful = deepfool_targeted(im, net, target)

    confidences_pert.append(confidence_pert)
    l2_norms.append(l2_norm)
    ssim_scores.append(ssim_value)
    iterations.append(loop_i)
    success.append(successful)

    labels = open(os.path.join('synset_words.txt'), 'r').read().split('\n')
    
    # Get the string label for the original and perturbed images
    str_label_orig = labels[np.int_(label_orig)].split(',')[0]
    str_label_pert = labels[np.int_(label_pert)].split(',')[0]

    def clip_tensor(A, minv, maxv):
        A = torch.max(A, minv*torch.ones(A.shape))
        A = torch.min(A, maxv*torch.ones(A.shape))
        return A

    clip = lambda x: clip_tensor(x, 0, 255)

    tf = transforms.Compose([transforms.Normalize(mean=[0, 0, 0], std=list(map(lambda x: 1 / x, std))),
    transforms.Normalize(mean=list(map(lambda x: -x, mean)), std=[1, 1, 1]),
    transforms.Lambda(clip),
    transforms.ToPILImage(),
    transforms.CenterCrop(224)])

    # Clip the perturbed image and convert it back to a PIL image
    pert_image = tf(pert_image.cpu()[0])

    # Save the perturbed image
    pert_image.save(os.path.join('./data/perturbed_images_alexnet', filename))
    
    # Print the labels of the original and perturbed images
    print(f"{filename}: Original label = {str_label_orig}, Perturbed label = {str_label_pert}")
    
    index += 1

    # Add model name as key and corresponding values as values in stats_dict_temp
    # Update test_dict with the new values for alexnet
    # Check if 'alexnet' is already in stats_dict_temp
    if 'alexnet' in stats_dict_temp:
        # If it is, update the existing dictionary for 'alexnet'
        stats_dict_temp['alexnet'].update({
            'confidences_pert': stats_dict_temp['alexnet'].get('confidences_pert', []) + [confidence_pert],
            'l2_norms': stats_dict_temp['alexnet'].get('l2_norms', []) + [l2_norm],
            'ssim_scores': stats_dict_temp['alexnet'].get('ssim_scores', []) + [ssim_value],
            'iterations': stats_dict_temp['alexnet'].get('iterations', []) + [loop_i],
            'success': stats_dict_temp['alexnet'].get('success', []) + [successful]
        })
    else:
        # If it isn't, add a new dictionary for 'alexnet'
        stats_dict_temp['alexnet'] = {
            'confidences_pert': [confidence_pert],
            'l2_norms': [l2_norm],
            'ssim_scores': [ssim_value],
            'iterations': [loop_i],
            'success': [successful]
        }
    
    with open('./stats/alexnet_stats_temp.json', 'w') as f:
        json.dump(stats_dict_temp, f)

In [ ]:
time =  time.process_time() - start
print("Total Time: ",time)

In [ ]:
# Add model name as key and corresponding values as values in stats_dict
stats_dict.update({'alexnet': {
                            'confidences_pert': confidences_pert,
                            'l2_norms': l2_norms,
                            'ssim_scores': ssim_scores,
                            'iterations': iterations,
                            'success': success
                            }})

In [ ]:
# Write dictionary to file
with open('./stats/alexnet_stats.json', 'w') as f:
    json.dump(stats_dict, f)

In [ ]:
# Read dictionary from file
with open('./stats/alexnet_stats.json', 'r') as f:
    stats_dict = json.load(f)

model_name = "alexnet"
stats = stats_dict[model_name]

print(f"Mean Confidence for {model_name}: {sum(stats['confidences_pert'])/len(stats['confidences_pert']):.4f}")
print(f"Mean l2_norm for {model_name}: {sum(stats['l2_norms'])/len(stats['l2_norms']):.4f}%")
print(f"Mean SSIM score for {model_name}: {sum(stats['ssim_scores'])/len(stats['ssim_scores']):.4f}")
print(f"Mean Iterations for {model_name}: {sum(stats['iterations'])/len(stats['iterations']):.4f}")
print(f"Mean success for {model_name}: {sum(stats['success'])/len(stats['success']):.4f}\n")

In [ ]:
# Shows results after excluding failed attacks

with open('./stats/alexnet_stats.json', 'r') as f:
    stats_dict = json.load(f)

model_name = "alexnet"
stats = stats_dict[model_name]

success_indices = [i for i, success in enumerate(stats['success']) if success]

success_confidences = [stats['confidences_pert'][i] for i in success_indices]
success_l2_norms = [stats['l2_norms'][i] for i in success_indices]
success_ssim_scores = [stats['ssim_scores'][i] for i in success_indices]
success_iterations = [stats['iterations'][i] for i in success_indices]

print(f"Mean Confidence for {model_name}: {sum(success_confidences) / len(success_confidences):.4f}")
print(f"Mean l2_norm for {model_name}: {sum(success_l2_norms) / len(success_l2_norms):.4f}%")
print(f"Mean SSIM score for {model_name}: {sum(success_ssim_scores) / len(success_ssim_scores):.4f}")
print(f"Mean Iterations for {model_name}: {math.ceil(sum(success_iterations) / len(success_iterations))}\n")


# **EfficientNetV2**

In [ ]:
import time
start = time.process_time()
stats_dict = {}
stats_dict_temp = {}

# Storing true labels in an array
filename = './data/LOC_val_solution/ILSVRC2012_val_true_label_indices.txt'

with open(filename, 'r') as file:
    true_labels = []
    for line in file:
        true_labels.append(int(line.strip()))

net = models.efficientnet_v2_s(pretrained=True)

# Switch to evaluation mode
net.eval()

img_dir = './data/ILSVRC2012_img_val/'

files = sorted(os.listdir(img_dir)) # Use this line if you want to run for the whole dataset

# Select only the specific file names 
# [1000:] = start from 1000 and end at file end, 
# [:1000] = run from start till 1000th file
# [65:36000] = run from 66th file to 36000th file

# files = os.listdir(img_dir)[36287:] #Change the value inside [] according to need

confidences_pert = []
l2_norms = []
ssim_scores = []
index = 0
iterations = []
success = []

for filename in files:
    
    # Create the 'perturbed_images' directory if it doesn't exist
    if not os.path.exists('./data/perturbed_images_efficientnet'):
        os.makedirs('./data/perturbed_images_efficientnet')

    # Load the image
    im_orig = Image.open(os.path.join(img_dir, filename)).convert('RGB')
    # im_orig = Image.open(os.path.join(img_dir, filename))
    # if im_orig.mode == 'L': # grayscale image
    #     im_orig = im_orig.convert('RGB')

    mean = [ 0.485, 0.456, 0.406 ]
    std = [ 0.229, 0.224, 0.225 ]

    # Apply the transformation pipeline
    im = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean = mean,
                             std = std)])(im_orig)
    
    # randomly select target class
    target = np.random.choice([i for i in range(1000) if i != true_labels[index]])

    # Perform the targeted deepfool attack
    r, loop_i, label_orig, label_pert, pert_image, confidences, confidence_pert, l2_norm, ssim_value, successful = deepfool_targeted(im, net, target)

    confidences_pert.append(confidence_pert)
    l2_norms.append(l2_norm)
    ssim_scores.append(ssim_value)
    iterations.append(loop_i)
    success.append(successful)

    labels = open(os.path.join('synset_words.txt'), 'r').read().split('\n')
    
    # Get the string label for the original and perturbed images
    str_label_orig = labels[np.int_(label_orig)].split(',')[0]
    str_label_pert = labels[np.int_(label_pert)].split(',')[0]

    def clip_tensor(A, minv, maxv):
        A = torch.max(A, minv*torch.ones(A.shape))
        A = torch.min(A, maxv*torch.ones(A.shape))
        return A

    clip = lambda x: clip_tensor(x, 0, 255)

    tf = transforms.Compose([transforms.Normalize(mean=[0, 0, 0], std=list(map(lambda x: 1 / x, std))),
    transforms.Normalize(mean=list(map(lambda x: -x, mean)), std=[1, 1, 1]),
    transforms.Lambda(clip),
    transforms.ToPILImage(),
    transforms.CenterCrop(224)])

    # Clip the perturbed image and convert it back to a PIL image
    pert_image = tf(pert_image.cpu()[0])

    # Save the perturbed image
    pert_image.save(os.path.join('./data/perturbed_images_efficientnet', filename))
    
    # Print the labels of the original and perturbed images
    print(f"{filename}: Original label = {str_label_orig}, Perturbed label = {str_label_pert}")
    
    index += 1

    # Add model name as key and corresponding values as values in stats_dict_temp
    # Update test_dict with the new values for alexnet
    # Check if 'efficientnet' is already in stats_dict_temp
    if 'efficientnet' in stats_dict_temp:
        # If it is, update the existing dictionary for 'efficientnet'
        stats_dict_temp['efficientnet'].update({
            'confidences_pert': stats_dict_temp['efficientnet'].get('confidences_pert', []) + [confidence_pert],
            'l2_norms': stats_dict_temp['efficientnet'].get('l2_norms', []) + [l2_norm],
            'ssim_scores': stats_dict_temp['efficientnet'].get('ssim_scores', []) + [ssim_value],
            'iterations': stats_dict_temp['efficientnet'].get('iterations', []) + [loop_i],
            'success': stats_dict_temp['efficientnet'].get('success', []) + [successful]
        })
    else:
        # If it isn't, add a new dictionary for 'efficientnet'
        stats_dict_temp['efficientnet'] = {
            'confidences_pert': [confidence_pert],
            'l2_norms': [l2_norm],
            'ssim_scores': [ssim_value],
            'iterations': [loop_i],
            'success': [successful]
        }
    
    with open('./stats/efficientnet_stats_temp.json', 'w') as f:
        json.dump(stats_dict_temp, f)

In [ ]:
time =  time.process_time() - start
print("Total Time: ",time)

In [ ]:
# Add model name as key and corresponding values as values in stats_dict
stats_dict.update({'efficientnet': {
                            'confidences_pert': confidences_pert,
                            'l2_norms': l2_norms,
                            'ssim_scores': ssim_scores,
                            'iterations': iterations,
                            'success': success
                            }})

In [ ]:
# Write dictionary to file
with open('./stats/efficientnet_stats.json', 'w') as f:
    json.dump(stats_dict, f)

In [ ]:
# Read dictionary from file
with open('./stats/efficientnet_stats.json', 'r') as f:
    stats_dict = json.load(f)

model_name = "efficientnet"
stats = stats_dict[model_name]

print(f"Mean Confidence for {model_name}: {sum(stats['confidences_pert'])/len(stats['confidences_pert']):.4f}")
print(f"Mean l2_norm for {model_name}: {sum(stats['l2_norms'])/len(stats['l2_norms']):.4f}%")
print(f"Mean SSIM score for {model_name}: {sum(stats['ssim_scores'])/len(stats['ssim_scores']):.4f}")
print(f"Mean Iterations for {model_name}: {sum(stats['iterations'])/len(stats['iterations']):.4f}")
print(f"Mean success for {model_name}: {sum(stats['success'])/len(stats['success']):.4f}\n")

In [ ]:
# Shows results after excluding failed attacks

with open('./stats/efficientnet_stats.json', 'r') as f:
    stats_dict = json.load(f)

model_name = "efficientnet"
stats = stats_dict[model_name]

success_indices = [i for i, success in enumerate(stats['success']) if success]

success_confidences = [stats['confidences_pert'][i] for i in success_indices]
success_l2_norms = [stats['l2_norms'][i] for i in success_indices]
success_ssim_scores = [stats['ssim_scores'][i] for i in success_indices]
success_iterations = [stats['iterations'][i] for i in success_indices]

print(f"Mean Confidence for {model_name}: {sum(success_confidences) / len(success_confidences):.4f}")
print(f"Mean l2_norm for {model_name}: {sum(success_l2_norms) / len(success_l2_norms):.4f}%")
print(f"Mean SSIM score for {model_name}: {sum(success_ssim_scores) / len(success_ssim_scores):.4f}")
print(f"Mean Iterations for {model_name}: {math.ceil(sum(success_iterations) / len(success_iterations))}\n")


# **GoogLeNet**

In [ ]:
import time
start = time.process_time()
stats_dict = {}
stats_dict_temp = {}

# Storing true labels in an array
filename = './data/LOC_val_solution/ILSVRC2012_val_true_label_indices.txt'

with open(filename, 'r') as file:
    true_labels = []
    for line in file:
        true_labels.append(int(line.strip()))

net = models.googlenet(pretrained=True)

# Switch to evaluation mode
net.eval()

img_dir = './data/ILSVRC2012_img_val/'

files = sorted(os.listdir(img_dir)) # Use this line if you want to run for the whole dataset

# Select only the specific file names 
# [1000:] = start from 1000 and end at file end, 
# [:1000] = run from start till 1000th file
# [65:36000] = run from 66th file to 36000th file

# files = sorted(os.listdir(img_dir))[19876:] #Change the value inside [] according to need

confidences_pert = []
l2_norms = []
ssim_scores = []
index = 0
iterations = []
success = []

for filename in files:
    
    # Create the 'perturbed_images' directory if it doesn't exist
    if not os.path.exists('./data/perturbed_images_googlenet'):
        os.makedirs('./data/perturbed_images_googlenet')

    # Load the image
    im_orig = Image.open(os.path.join(img_dir, filename)).convert('RGB')
    # im_orig = Image.open(os.path.join(img_dir, filename))
    # if im_orig.mode == 'L': # grayscale image
    #     im_orig = im_orig.convert('RGB')

    mean = [ 0.485, 0.456, 0.406 ]
    std = [ 0.229, 0.224, 0.225 ]

    # Apply the transformation pipeline
    im = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean = mean,
                             std = std)])(im_orig)
    
    # randomly select target class
    target = np.random.choice([i for i in range(1000) if i != true_labels[index]])

    # Perform the targeted deepfool attack
    r, loop_i, label_orig, label_pert, pert_image, confidences, confidence_pert, l2_norm, ssim_value, successful = deepfool_targeted(im, net, target)

    confidences_pert.append(confidence_pert)
    l2_norms.append(l2_norm)
    ssim_scores.append(ssim_value)
    iterations.append(loop_i)
    success.append(successful)

    labels = open(os.path.join('synset_words.txt'), 'r').read().split('\n')
    
    # Get the string label for the original and perturbed images
    str_label_orig = labels[np.int_(label_orig)].split(',')[0]
    str_label_pert = labels[np.int_(label_pert)].split(',')[0]

    def clip_tensor(A, minv, maxv):
        A = torch.max(A, minv*torch.ones(A.shape))
        A = torch.min(A, maxv*torch.ones(A.shape))
        return A

    clip = lambda x: clip_tensor(x, 0, 255)

    tf = transforms.Compose([transforms.Normalize(mean=[0, 0, 0], std=list(map(lambda x: 1 / x, std))),
    transforms.Normalize(mean=list(map(lambda x: -x, mean)), std=[1, 1, 1]),
    transforms.Lambda(clip),
    transforms.ToPILImage(),
    transforms.CenterCrop(224)])

    # Clip the perturbed image and convert it back to a PIL image
    pert_image = tf(pert_image.cpu()[0])

    # Save the perturbed image
    pert_image.save(os.path.join('./data/perturbed_images_googlenet', filename))
    
    # Print the labels of the original and perturbed images
    print(f"{filename}: Original label = {str_label_orig}, Perturbed label = {str_label_pert}")
    
    index += 1

    # Add model name as key and corresponding values as values in stats_dict_temp
    # Update test_dict with the new values for alexnet
    # Check if 'googlenet' is already in stats_dict_temp
    if 'googlenet' in stats_dict_temp:
        # If it is, update the existing dictionary for 'googlenet'
        stats_dict_temp['googlenet'].update({
            'confidences_pert': stats_dict_temp['googlenet'].get('confidences_pert', []) + [confidence_pert],
            'l2_norms': stats_dict_temp['googlenet'].get('l2_norms', []) + [l2_norm],
            'ssim_scores': stats_dict_temp['googlenet'].get('ssim_scores', []) + [ssim_value],
            'iterations': stats_dict_temp['googlenet'].get('iterations', []) + [loop_i],
            'success': stats_dict_temp['googlenet'].get('success', []) + [successful]
        })
    else:
        # If it isn't, add a new dictionary for 'googlenet'
        stats_dict_temp['googlenet'] = {
            'confidences_pert': [confidence_pert],
            'l2_norms': [l2_norm],
            'ssim_scores': [ssim_value],
            'iterations': [loop_i],
            'success': [successful]
        }
    
    with open('./stats/googlenet_stats_temp.json', 'w') as f:
        json.dump(stats_dict_temp, f)


In [ ]:
# Add model name as key and corresponding values as values in stats_dict
stats_dict.update({'googlenet': {
                            'confidences_pert': confidences_pert,
                            'l2_norms': l2_norms,
                            'ssim_scores': ssim_scores,
                            'iterations': iterations,
                            'success': success
                            }})

In [ ]:
# Write dictionary to file
with open('./stats/googlenet_stats.json', 'w') as f:
    json.dump(stats_dict, f)

In [ ]:
# Read dictionary from file
with open('./stats/googlenet_stats.json', 'r') as f:
    stats_dict = json.load(f)

model_name = "googlenet"
stats = stats_dict[model_name]

print(f"Mean Confidence for {model_name}: {sum(stats['confidences_pert'])/len(stats['confidences_pert']):.4f}")
print(f"Mean l2_norm for {model_name}: {sum(stats['l2_norms'])/len(stats['l2_norms']):.4f}%")
print(f"Mean SSIM score for {model_name}: {sum(stats['ssim_scores'])/len(stats['ssim_scores']):.4f}")
print(f"Mean Iterations for {model_name}: {sum(stats['iterations'])/len(stats['iterations']):.4f}")
print(f"Mean success for {model_name}: {sum(stats['success'])/len(stats['success']):.4f}\n")

In [ ]:
# Shows results after excluding failed attacks

with open('./stats/googlenet_stats.json', 'r') as f:
    stats_dict = json.load(f)

model_name = "googlenet"
stats = stats_dict[model_name]

success_indices = [i for i, success in enumerate(stats['success']) if success]

success_confidences = [stats['confidences_pert'][i] for i in success_indices]
success_l2_norms = [stats['l2_norms'][i] for i in success_indices]
success_ssim_scores = [stats['ssim_scores'][i] for i in success_indices]
success_iterations = [stats['iterations'][i] for i in success_indices]

print(f"Mean Confidence for {model_name}: {sum(success_confidences) / len(success_confidences):.4f}")
print(f"Mean l2_norm for {model_name}: {sum(success_l2_norms) / len(success_l2_norms):.4f}%")
print(f"Mean SSIM score for {model_name}: {sum(success_ssim_scores) / len(success_ssim_scores):.4f}")
print(f"Mean Iterations for {model_name}: {math.ceil(sum(success_iterations) / len(success_iterations))}\n")


# **InceptionV3**

In [ ]:
import time
start = time.process_time()
stats_dict = {}
stats_dict_temp = {}

# Storing true labels in an array
filename = './data/LOC_val_solution/ILSVRC2012_val_true_label_indices.txt'

with open(filename, 'r') as file:
    true_labels = []
    for line in file:
        true_labels.append(int(line.strip()))

net = models.inception_v3(pretrained=True)

# Switch to evaluation mode
net.eval()

img_dir = './data/ILSVRC2012_img_val/'

# files = sorted(os.listdir(img_dir)) # Use this line if you want to run for the whole dataset

# Select only the specific file names 
# [1000:] = start from 1000 and end at file end, 
# [:1000] = run from start till 1000th file
# [65:36000] = run from 66th file to 36000th file

files = sorted(os.listdir(img_dir))[36306:] #Change the value inside [] according to need

confidences_pert = []
l2_norms = []
ssim_scores = []
index = 0
iterations = []
success = []

for filename in files:
    
    # Create the 'perturbed_images' directory if it doesn't exist
    if not os.path.exists('./data/perturbed_images_inceptionV3'):
        os.makedirs('./data/perturbed_images_inceptionV3')

    # Load the image
    im_orig = Image.open(os.path.join(img_dir, filename)).convert('RGB')
    # im_orig = Image.open(os.path.join(img_dir, filename))
    # if im_orig.mode == 'L': # grayscale image
    #     im_orig = im_orig.convert('RGB')

    mean = [ 0.485, 0.456, 0.406 ]
    std = [ 0.229, 0.224, 0.225 ]

    # Apply the transformation pipeline
    im = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.Resize((299, 299)),
        transforms.ToTensor(),
        transforms.Normalize(mean = mean,
                             std = std)])(im_orig)
    
    # randomly select target class
    target = np.random.choice([i for i in range(1000) if i != true_labels[index]])

    # Perform the targeted deepfool attack
    r, loop_i, label_orig, label_pert, pert_image, confidences, confidence_pert, l2_norm, ssim_value, successful = deepfool_targeted(im, net, target)

    confidences_pert.append(confidence_pert)
    l2_norms.append(l2_norm)
    ssim_scores.append(ssim_value)
    iterations.append(loop_i)
    success.append(successful)

    labels = open(os.path.join('synset_words.txt'), 'r').read().split('\n')
    
    # Get the string label for the original and perturbed images
    str_label_orig = labels[np.int_(label_orig)].split(',')[0]
    str_label_pert = labels[np.int_(label_pert)].split(',')[0]

    def clip_tensor(A, minv, maxv):
        A = torch.max(A, minv*torch.ones(A.shape))
        A = torch.min(A, maxv*torch.ones(A.shape))
        return A

    clip = lambda x: clip_tensor(x, 0, 255)

    tf = transforms.Compose([transforms.Normalize(mean=[0, 0, 0], std=list(map(lambda x: 1 / x, std))),
    transforms.Normalize(mean=list(map(lambda x: -x, mean)), std=[1, 1, 1]),
    transforms.Lambda(clip),
    transforms.ToPILImage(),
    transforms.CenterCrop(224)])

    # Clip the perturbed image and convert it back to a PIL image
    pert_image = tf(pert_image.cpu()[0])

    # Save the perturbed image
    pert_image.save(os.path.join('./data/perturbed_images_inceptionV3', filename))
    
    # Print the labels of the original and perturbed images
    print(f"{filename}: Original label = {str_label_orig}, Perturbed label = {str_label_pert}")
    
    index += 1

    # Add model name as key and corresponding values as values in stats_dict_temp
    # Update test_dict with the new values for alexnet
    # Check if 'inceptionV3' is already in stats_dict_temp
    if 'inceptionV3' in stats_dict_temp:
        # If it is, update the existing dictionary for 'inceptionV3'
        stats_dict_temp['inceptionV3'].update({
            'confidences_pert': stats_dict_temp['inceptionV3'].get('confidences_pert', []) + [confidence_pert],
            'l2_norms': stats_dict_temp['inceptionV3'].get('l2_norms', []) + [l2_norm],
            'ssim_scores': stats_dict_temp['inceptionV3'].get('ssim_scores', []) + [ssim_value],
            'iterations': stats_dict_temp['inceptionV3'].get('iterations', []) + [loop_i],
            'success': stats_dict_temp['inceptionV3'].get('success', []) + [successful]
        })
    else:
        # If it isn't, add a new dictionary for 'inceptionV3'
        stats_dict_temp['inceptionV3'] = {
            'confidences_pert': [confidence_pert],
            'l2_norms': [l2_norm],
            'ssim_scores': [ssim_value],
            'iterations': [loop_i],
            'success': [successful]
        }
    
    with open('./stats/inceptionV3_stats_temp.json', 'w') as f:
        json.dump(stats_dict_temp, f)

In [ ]:
# Add model name as key and corresponding values as values in stats_dict
stats_dict.update({'inceptionV3': {
                            'confidences_pert': confidences_pert,
                            'l2_norms': l2_norms,
                            'ssim_scores': ssim_scores,
                            'iterations': iterations,
                            'success': success
                            }})

In [ ]:
# Write dictionary to file
with open('./stats/inceptionV3_stats.json', 'w') as f:
    json.dump(stats_dict, f)

In [ ]:
# Read dictionary from file
with open('./stats/inceptionV3_stats.json', 'r') as f:
    stats_dict = json.load(f)

model_name = "inceptionV3"
stats = stats_dict[model_name]

print(f"Mean Confidence for {model_name}: {sum(stats['confidences_pert'])/len(stats['confidences_pert']):.4f}")
print(f"Mean l2_norm for {model_name}: {sum(stats['l2_norms'])/len(stats['l2_norms']):.4f}%")
print(f"Mean SSIM score for {model_name}: {sum(stats['ssim_scores'])/len(stats['ssim_scores']):.4f}")
print(f"Mean Iterations for {model_name}: {sum(stats['iterations'])/len(stats['iterations']):.4f}")
print(f"Mean success for {model_name}: {sum(stats['success'])/len(stats['success']):.4f}\n")

In [ ]:
# Shows results after excluding failed attacks

with open('./stats/inceptionV3_stats.json', 'r') as f:
    stats_dict = json.load(f)

model_name = "inceptionV3"
stats = stats_dict[model_name]

success_indices = [i for i, success in enumerate(stats['success']) if success]

success_confidences = [stats['confidences_pert'][i] for i in success_indices]
success_l2_norms = [stats['l2_norms'][i] for i in success_indices]
success_ssim_scores = [stats['ssim_scores'][i] for i in success_indices]
success_iterations = [stats['iterations'][i] for i in success_indices]

print(f"Mean Confidence for {model_name}: {sum(success_confidences) / len(success_confidences):.4f}")
print(f"Mean l2_norm for {model_name}: {sum(success_l2_norms) / len(success_l2_norms):.4f}%")
print(f"Mean SSIM score for {model_name}: {sum(success_ssim_scores) / len(success_ssim_scores):.4f}")
print(f"Mean Iterations for {model_name}: {math.ceil(sum(success_iterations) / len(success_iterations))}\n")


# **VisionTransformer**

In [ ]:
import time
start = time.process_time()
stats_dict = {}
stats_dict_temp = {}

# Storing true labels in an array
filename = './data/LOC_val_solution/ILSVRC2012_val_true_label_indices.txt'

with open(filename, 'r') as file:
    true_labels = []
    for line in file:
        true_labels.append(int(line.strip()))

net = models.vit_b_16(pretrained=True)

# Switch to evaluation mode
net.eval()

img_dir = './data/ILSVRC2012_img_val/'

files = sorted(os.listdir(img_dir)) # Use this line if you want to run for the whole dataset

# Select only the specific file names 
# [1000:] = start from 1000 and end at file end, 
# [:1000] = run from start till 1000th file
# [65:36000] = run from 66th file to 36000th file

# files = sorted(os.listdir(img_dir))[19876:] #Change the value inside [] according to need

confidences_pert = []
l2_norms = []
ssim_scores = []
index = 0
iterations = []
success = []

for filename in files:
    
    # Create the 'perturbed_images' directory if it doesn't exist
    if not os.path.exists('./data/perturbed_images_visionTransformer'):
        os.makedirs('./data/perturbed_images_visionTransformer')

    # Load the image
    im_orig = Image.open(os.path.join(img_dir, filename)).convert('RGB')
    # im_orig = Image.open(os.path.join(img_dir, filename))
    # if im_orig.mode == 'L': # grayscale image
    #     im_orig = im_orig.convert('RGB')

    mean = [ 0.485, 0.456, 0.406 ]
    std = [ 0.229, 0.224, 0.225 ]

    # Apply the transformation pipeline
    im = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean = mean,
                             std = std)])(im_orig)
    
    # randomly select target class
    target = np.random.choice([i for i in range(1000) if i != true_labels[index]])

    # Perform the targeted deepfool attack
    r, loop_i, label_orig, label_pert, pert_image, confidences, confidence_pert, l2_norm, ssim_value, successful = deepfool_targeted(im, net, target)

    confidences_pert.append(confidence_pert)
    l2_norms.append(l2_norm)
    ssim_scores.append(ssim_value)
    iterations.append(loop_i)
    success.append(successful)

    labels = open(os.path.join('synset_words.txt'), 'r').read().split('\n')
    
    # Get the string label for the original and perturbed images
    str_label_orig = labels[np.int_(label_orig)].split(',')[0]
    str_label_pert = labels[np.int_(label_pert)].split(',')[0]

    def clip_tensor(A, minv, maxv):
        A = torch.max(A, minv*torch.ones(A.shape))
        A = torch.min(A, maxv*torch.ones(A.shape))
        return A

    clip = lambda x: clip_tensor(x, 0, 255)

    tf = transforms.Compose([transforms.Normalize(mean=[0, 0, 0], std=list(map(lambda x: 1 / x, std))),
    transforms.Normalize(mean=list(map(lambda x: -x, mean)), std=[1, 1, 1]),
    transforms.Lambda(clip),
    transforms.ToPILImage(),
    transforms.CenterCrop(224)])

    # Clip the perturbed image and convert it back to a PIL image
    pert_image = tf(pert_image.cpu()[0])

    # Save the perturbed image
    pert_image.save(os.path.join('./data/perturbed_images_visionTransformer', filename))
    
    # Print the labels of the original and perturbed images
    print(f"{filename}: Original label = {str_label_orig}, Perturbed label = {str_label_pert}")
    
    index += 1

    # Add model name as key and corresponding values as values in stats_dict
    # Update test_dict with the new values for alexnet
    # Check if 'visionTransformer' is already in stats_dict_temp
    if 'visionTransformer' in stats_dict_temp:
        # If it is, update the existing dictionary for 'visionTransformer'
        stats_dict_temp['visionTransformer'].update({
            'confidences_pert': stats_dict_temp['visionTransformer'].get('confidences_pert', []) + [confidence_pert],
            'l2_norms': stats_dict_temp['visionTransformer'].get('l2_norms', []) + [l2_norm],
            'ssim_scores': stats_dict_temp['visionTransformer'].get('ssim_scores', []) + [ssim_value],
            'iterations': stats_dict_temp['visionTransformer'].get('iterations', []) + [loop_i],
            'success': stats_dict_temp['visionTransformer'].get('success', []) + [successful]
        })
    else:
        # If it isn't, add a new dictionary for 'visionTransformer'
        stats_dict_temp['visionTransformer'] = {
            'confidences_pert': [confidence_pert],
            'l2_norms': [l2_norm],
            'ssim_scores': [ssim_value],
            'iterations': [loop_i],
            'success': [successful]
        }
    
    with open('./stats/visionTransformer_stats_temp_.json', 'w') as f:
        json.dump(stats_dict_temp, f)


In [ ]:
time =  time.process_time() - start
print("Total Time: ",time)

In [ ]:
# Add model name as key and corresponding values as values in stats_dict
stats_dict.update({'visionTransformer': {
                            'confidences_pert': confidences_pert,
                            'l2_norms': l2_norms,
                            'ssim_scores': ssim_scores,
                            'iterations': iterations,
                            'success': success
                            }})

In [ ]:
# Write dictionary to file
with open('./stats/visionTransformer_stats.json', 'w') as f:
    json.dump(stats_dict, f)

In [ ]:
# Read dictionary from file
with open('./stats/visionTransformer_stats.json', 'r') as f:
    stats_dict = json.load(f)

model_name = "visionTransformer"
stats = stats_dict[model_name]

print(f"Mean Confidence for {model_name}: {sum(stats['confidences_pert'])/len(stats['confidences_pert']):.4f}")
print(f"Mean l2_norm for {model_name}: {sum(stats['l2_norms'])/len(stats['l2_norms']):.4f}%")
print(f"Mean SSIM score for {model_name}: {sum(stats['ssim_scores'])/len(stats['ssim_scores']):.4f}")
print(f"Mean Iterations for {model_name}: {sum(stats['iterations'])/len(stats['iterations']):.4f}")
print(f"Mean success for {model_name}: {sum(stats['success'])/len(stats['success']):.4f}\n")

In [ ]:
# Shows results after excluding failed attacks

with open('./stats/visionTransformer_stats.json', 'r') as f:
    stats_dict = json.load(f)

model_name = "visionTransformer"
stats = stats_dict[model_name]

success_indices = [i for i, success in enumerate(stats['success']) if success]

success_confidences = [stats['confidences_pert'][i] for i in success_indices]
success_l2_norms = [stats['l2_norms'][i] for i in success_indices]
success_ssim_scores = [stats['ssim_scores'][i] for i in success_indices]
success_iterations = [stats['iterations'][i] for i in success_indices]

print(f"Mean Confidence for {model_name}: {sum(success_confidences) / len(success_confidences):.4f}")
print(f"Mean l2_norm for {model_name}: {sum(success_l2_norms) / len(success_l2_norms):.4f}%")
print(f"Mean SSIM score for {model_name}: {sum(success_ssim_scores) / len(success_ssim_scores):.4f}")
print(f"Mean Iterations for {model_name}: {math.ceil(sum(success_iterations) / len(success_iterations))}\n")